### Importing the libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Importing the MNIST dataset

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

### Preprocessing the data

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image=tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

batch_size = 150
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model

### Outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 500

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
#     tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
#     tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
#     tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
#     tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

In [5]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)

# model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [6]:
num_epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(
    train_data,
    epochs=num_epochs,
    callbacks = [early_stopping],
    validation_data=(validation_inputs, validation_targets),
    verbose=2)

Epoch 1/10
360/360 - 6s - loss: 0.2375 - accuracy: 0.9295 - val_loss: 0.1026 - val_accuracy: 0.9687 - 6s/epoch - 18ms/step
Epoch 2/10
360/360 - 5s - loss: 0.0867 - accuracy: 0.9732 - val_loss: 0.0704 - val_accuracy: 0.9782 - 5s/epoch - 14ms/step
Epoch 3/10
360/360 - 6s - loss: 0.0552 - accuracy: 0.9824 - val_loss: 0.0568 - val_accuracy: 0.9800 - 6s/epoch - 16ms/step
Epoch 4/10
360/360 - 6s - loss: 0.0396 - accuracy: 0.9871 - val_loss: 0.0414 - val_accuracy: 0.9873 - 6s/epoch - 16ms/step
Epoch 5/10
360/360 - 6s - loss: 0.0307 - accuracy: 0.9899 - val_loss: 0.0288 - val_accuracy: 0.9892 - 6s/epoch - 16ms/step
Epoch 6/10
360/360 - 6s - loss: 0.0248 - accuracy: 0.9916 - val_loss: 0.0240 - val_accuracy: 0.9923 - 6s/epoch - 16ms/step
Epoch 7/10
360/360 - 6s - loss: 0.0181 - accuracy: 0.9943 - val_loss: 0.0191 - val_accuracy: 0.9942 - 6s/epoch - 16ms/step
Epoch 8/10
360/360 - 6s - loss: 0.0197 - accuracy: 0.9936 - val_loss: 0.0319 - val_accuracy: 0.9890 - 6s/epoch - 16ms/step
Epoch 9/10
360/3

### Testing the model

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 633ms/step - loss: 0.0746 - accuracy: 0.9806


In [8]:
print('Test loss:{0:.2f}. Test accuracy:{1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss:0.07. Test accuracy:98.06%
